In [35]:
!pip install --user ncon

In [51]:
import numpy as np
from ncon import ncon
# import networkx as nx

PLUS = np.array([1, 1]) * .5 ** .5
PP = np.kron(PLUS, PLUS)
BELL = np.array([1, 0, 0, 1]) * .5 ** .5
T = np.kron(BELL, BELL).reshape(2, 2, 2, 2)
eps = np.array([[0, 1], [-1, 0]])

In [47]:
# contracting state measured in it's basis
ncon((BELL, BELL), [[1], [1]])

array(1.)

In [48]:
# compute matrix determinant
ncon((BELL.reshape(2, 2).T, eps, BELL.reshape(2, 2)), [[-1, 1], [1, 2], [2, -2]])

array([[ 0. ,  0.5],
       [-0.5,  0. ]])

In [49]:
# compute concurrence of a bell state
ncon((BELL.reshape(2, 2), eps, eps, BELL.reshape(2, 2)), [[1, 2], [1, 3], [2, 4], [3, 4]])

array(1.)

In [50]:
# compute concurrence of |++> state
ncon((PP.reshape(2, 2), eps, eps, PP.reshape(2, 2)), [[1, 2], [1, 3], [2, 4], [3, 4]])

array(0.)

In [57]:
# Contact double-bell state
BB = np.kron(BELL, BELL).reshape(2, 2, 2, 2)

ncon((BB, eps, eps, eps, eps, BB), [
    [1, 2, 3, 4], # BB
    [1, 5],  # eps0
    [2, 6], # eps1
    [3, 7], # eps2
    [4, 8], # eps4
    [5, 6, 7, 8]# BB
])

array(1.)